Find Candiate molecule through FP match


In [1]:
from rdkit import Chem
from rdkit.DataStructs import FingerprintSimilarity as tanimoto
from rdkit.Chem import Draw
from rdkit.Chem import AllChem,MACCSkeys
import pandas as pd
from ast import literal_eval
from joblib import Parallel, delayed

In [2]:
def Similarity(smiles,target):
    #smiles is a list of smiles
    try:
        fp=MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles(smiles))
        score=tanimoto(fp,target)
    except:
        score=0
            
    return score

In [3]:
df=pd.read_csv('FinalSmiles.csv',header=0)
molecules=df['reactants'].tolist()
print(len(molecules))
molecules.extend(df['products'].tolist())
print(len(molecules))
molecules.extend(df['spectators'].tolist())
len(molecules)

C:\Users\ks\AppData\Local\Temp\ipykernel_10216\3729939481.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('FinalSmiles.csv',header=0)


1808254
3616508


5424762

In [13]:
targetmol=Chem.MolFromSmarts('C1C2C(C(C(C(O1)O2)O)O)O')
#targetmol=Chem.MolFromSmarts('C1=CC=C(C=C1)CCOC2=CC=CC=C2')
Chem.SanitizeMol(targetmol)
target=MACCSkeys.GenMACCSKeys(targetmol)

In [7]:

print(target)
score=Parallel(n_jobs=-1,verbose=1)(delayed(Similarity)(smiles,target) for smiles in molecules)
pd.DataFrame({'Molecules':molecules,'Similarity':score}).to_csv('BioPC2_cresol_MACCS_Similarity.csv')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 1420 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 16148 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 45564 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 88764 tasks      | elapsed:   26.1s
[Parallel(n_jobs=-1)]: Done 141564 tasks      | elapsed:   39.3s
[Parallel(n_jobs=-1)]: Done 203964 tasks      | elapsed:   55.5s
[Parallel(n_jobs=-1)]: Done 275964 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 357564 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 448764 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 549564 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 659964 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 779964 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 909564 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]

In [19]:
pd.DataFrame({'Molecules':molecules,'Similarity':score}).sort_values(by='Similarity', ascending=False).to_csv('BioPC1_MACCS_Similarity.csv')

In [10]:
#DO THIS AFTER SUBSTRUCT SIMILARITY

df=pd.read_csv('Biomass/PC SubHits/BioPC5_Substruct_Similarity_new.csv',header=0)
mols=df['Molecules'].tolist()
substruct=df['Similarity'].tolist()


In [11]:
matchmol=[]
for i,m in enumerate(mols):
    if substruct[i]>=1:
        matchmol.append(m)
    else:
        continue

In [14]:
score=Parallel(n_jobs=-1,verbose=1)(delayed(Similarity)(smiles,target) for smiles in matchmol) 
pd.DataFrame({'Molecules':matchmol,'Similarity':score}).sort_values(by='Similarity', ascending=False).to_csv('Biomass/PC SubHits/BioPC5SubFP_Similarity_LG.csv')


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 3560 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 19560 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 41960 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Done 70760 tasks      | elapsed:   22.0s
[Parallel(n_jobs=-1)]: Done 105960 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done 147560 tasks      | elapsed:   45.2s
[Parallel(n_jobs=-1)]: Done 195560 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 249960 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 310760 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 377960 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 451560 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 531560 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 617960 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]

In [21]:
similar_df=pd.DataFrame({'Molecules':matchmol,'Similarity':score})
sorted_df=similar_df.sort_values(by='Similarity', ascending=False)
print(sorted_df['Molecule'])

1210299    [CH:19]1[C:18]2[C:12]3[CH:11]=[CH:10][CH:9]=[C...
1210298    [O:16]1[CH:20]=[CH:19][C:18]2[CH:21]=[C:2]3[C:...
1214369    [C:29]([O:3][C@@H:2]1[CH:4]=[CH:5][C@H:6]2[CH2...
1163639    [CH3:17][C:13]1[O:12][C:11](=[O:18])[CH:10]([C...
1665023    [O:17]1[CH:21]=[CH:20][CH:19]=[C:18]1[C:22]([O...
1102414    [C:35]([O:12][C@@H:13]1[C@@H:14]([O:15][CH2:16...
1514832    [O:25]=[C:24]1[C@H:10]([O:9][C:7](=[O:8])[C:4]...
1319155    [OH:18][CH:17]1[C:15]2[CH:14]=[CH:9][CH:8]=[CH...
1516371    [O:25]=[C:24]1[C@H:10]([O:9][C:7](=[O:8])[C:4]...
1463576    [O:25]=[C:24]1[C@H:10]([O:9][C:7](=[O:8])[C:4]...
Name: Molecules, dtype: object